import required package

In [ ]:
import binascii
import struct
import json
import zarr
from ncplib.packets import decode_packet
import xarray as xr
import numpy as np


open the file and read into variable data

In [ ]:
#i renamed the pbd2 file
with open("RFEYE.ncp", "rb") as f:
    data = f.read()
f.close()

This is a function that gets the starts and stops of each ncp packet and saves them as list of tuples. Example: [(4, 26305), (26309, 52611)]

In [ ]:
def get_slices(b):
    pos = 0
    slices=[]
    while pos < len(b):
        msg_len = struct.unpack('<L', b[pos+8:pos+12])[0]*4
        slices.append((pos, pos+msg_len))
        pos = pos+msg_len
    return slices

Get the slices

In [ ]:
slices = get_slices(data)

converts the tuple returned by decode_packet to a dictionary

In [ ]:
def tup_to_dict(*args):
    packet_type, packet_id, timestamp, source_id, fields = args
    packet_dict = {'packet_id': packet_id, 'timestamp': timestamp, 'source_id': int(binascii.hexlify(source_id))}
    field_dict = {}
    for arg in fields:
        data_dict = {}
        name, _id, data = arg
        for field in data:
            key, value = field
            data_dict[key]=value
        field_dict[name]=data_dict
        packet_dict['fields'] = field_dict
    return packet_dict

Function to read dictionary to xarray DataArray

In [ ]:
def dict_to_xr(_dict):
    source_id = _dict['source_id']
    rlev = _dict['fields']['SWEP']['RLEV']
    pdat = np.array(_dict['fields']['SWEP']['PDAT'])/2 - 127.5 + rlev
    start_frq = _dict['fields']['SWEP']['FSTA']+ _dict['fields']['SWEP']['FSAM']/1000000000
    stop_frq = _dict['fields']['SWEP']['FSTP']+ _dict['fields']['SWEP']['FSPM']/1000000000
    timestamp = _dict['fields']['SWEP']['UTIM'] + _dict['fields']['SWEP']['NANO']/1000000000
    samp = _dict['fields']['SWEP']['SAMP']
    rbw = _dict['fields']['SWEP']['RESB']
    freqs=np.linspace(start_frq,stop_frq,samp)
    return xr.DataArray.from_dict({
        "coords": {
        "freq": {"dims": "freq", "data": freqs, "attrs": {"units": "hz"}},
        "time": {"dims": "time", "data": [timestamp], "attrs": {"units": "s"}}
        },
        'attrs': {'source_id': source_id,'rlev': rlev, 'start_frq': start_frq, 'stop_frq': stop_frq, 'samp': samp, 'rbw': rbw},
        'data': [pdat],
        "dims": ("time","freq"),
        'name': 'power'})

use the slices to convert to xarrays and save to a dictionary then combine the ones with the same source and start stop frequency to a single xarray and save the zarr file

In [ ]:
import time
start_time = time.time()
scans = []
data_dict = {}

for s in slices:
    start, stop = s
    arr = dict_to_xr(tup_to_dict(*decode_packet(data[start:stop])))
    source = arr.attrs['source_id']
    start_frq = arr.attrs['start_frq']
    stop_frq = arr.attrs['stop_frq']
    scan_name = f'{source}-{start_frq}_{stop_frq}'
    if scan_name in scans:
        data_dict[scan_name].append(arr)
    else:
        scans.append(scan_name)
        data_dict[scan_name] = [arr]
for scan in scans:
    combined = xr.combine_by_coords(data_dict[scan], combine_attrs = "drop_conflicts")
    store = zarr.ZipStore(f'{scan}.zip', mode='w')
    combined.to_zarr(store=store)
stop_time = time.time()
print(stop_time-start_time)

In [ ]:
import os
zips = []
files = os.listdir()
for file in files:
    if file[-3:] == 'zip':
        zips.append(file)
zips

In [ ]:
xr_data = zips[0]

In [ ]:
xr_data = xr.open_zarr(zips[2])

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.Image(xr_data).opts(cmap='jet', clim=(-100,-75), width=1000)